In [7]:
# Loan Default Prediction with SHAP Analysis
# Using 3:1 undersampling ratio to improve detection ability for the default class

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve, f1_score, recall_score
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
import shap  # Add SHAP library
import warnings
warnings.filterwarnings('ignore')

# 1. Load Data
print("Loading data...")
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")

# 2. Data Exploration and Preprocessing
print("\nData preprocessing...")

# 2.1 Check missing values
print("Checking missing values...")
print("Training set missing values:")
print(train_df.isnull().sum())
print("\nTest set missing values:")
print(test_df.isnull().sum())

# 2.2 Check data distribution
if 'Default' in train_df.columns:
    print("\nTarget variable distribution:")
    default_counts = train_df['Default'].value_counts(normalize=True) * 100
    print(f"Non-default (0): {default_counts[0]:.2f}%")
    print(f"Default (1): {default_counts[1]:.2f}%")
    
    # Calculate positive-negative ratio for handling imbalance
    neg, pos = np.bincount(train_df['Default'])
    print(f"Positive samples: {pos}, Negative samples: {neg}, Ratio: 1:{neg/pos:.2f}")

# 2.3 Extract features and target variable
if 'Default' in train_df.columns:
    X = train_df.drop('Default', axis=1)
    y = train_df['Default']
else:
    X = train_df.copy()
    y = None
    print("Warning: 'Default' column not found in training set")

# Save LoanID for later use
loan_ids = None
if 'LoanID' in X.columns:
    loan_ids = X['LoanID'].copy()

# 2.4 Split training and validation sets - use stratified sampling to ensure consistent default ratio
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training subset shape: {X_train.shape}")
print(f"Validation subset shape: {X_val.shape}")

# Handle label column in test set if it exists
test_loan_ids = None
if 'LoanID' in test_df.columns:
    test_loan_ids = test_df['LoanID'].copy()
    X_test = test_df.drop('LoanID', axis=1)
    if 'Default' in test_df.columns:
        y_test = test_df['Default']
        X_test = X_test.drop('Default', axis=1)
    else:
        X_test = test_df.copy()
        y_test = None
else:
    X_test = test_df.copy()
    if 'Default' in X_test.columns:
        y_test = X_test['Default']
        X_test = X_test.drop('Default', axis=1)
    else:
        y_test = None

# 2.5 Handle ID columns (if they exist)
id_cols = [col for col in X_train.columns if 'ID' in col or 'Id' in col]
for col in id_cols:
    if col in X_train.columns:
        X_train = X_train.drop(col, axis=1)
    if col in X_val.columns:
        X_val = X_val.drop(col, axis=1)
    if col in X_test.columns:
        X_test = X_test.drop(col, axis=1)

# 2.6 Handle categorical variables
print("\nProcessing categorical variables...")
categorical_cols = [
    "Education",
    "EmploymentType",
    "MaritalStatus",
    "HasMortgage",
    "HasDependents",
    "LoanPurpose", 
    "HasCoSigner"
]

# Check if all categorical variables are in the dataset
for col in categorical_cols[:]:  # Use slice to create copy to avoid modifying during iteration
    if col not in X_train.columns:
        categorical_cols.remove(col)
        print(f"Warning: Column '{col}' not in training set")

# Use LabelEncoder for categorical variables
label_encoders = {}
for col in categorical_cols:
    if col in X_train.columns:
        le = LabelEncoder()
        X_train[col] = le.fit_transform(X_train[col])
        
        # Ensure validation and test sets use the same encoding
        if col in X_val.columns:
            try:
                X_val[col] = le.transform(X_val[col])
            except ValueError:
                print(f"Warning: Validation set column '{col}' contains categories not seen in training. Will replace with most common category.")
                most_frequent = le.transform([X_train[col].value_counts().index[0]])[0]
                X_val[col] = X_val[col].map(lambda x: most_frequent if x not in le.classes_ else le.transform([x])[0])
        
        if col in X_test.columns:
            try:
                X_test[col] = le.transform(X_test[col])
            except ValueError:
                print(f"Warning: Test set column '{col}' contains categories not seen in training. Will replace with most common category.")
                most_frequent = le.transform([X_train[col].value_counts().index[0]])[0]
                X_test[col] = X_test[col].map(lambda x: most_frequent if x not in le.classes_ else le.transform([x])[0])
        
        label_encoders[col] = le
        print(f"Column '{col}' encoded, category mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# 2.7 Fill missing values
print("\nHandling missing values...")
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Use median for numeric features
if len(numeric_cols) > 0:
    num_imputer = SimpleImputer(strategy='median')
    X_train[numeric_cols] = num_imputer.fit_transform(X_train[numeric_cols])
    X_val[numeric_cols] = num_imputer.transform(X_val[numeric_cols])
    X_test[numeric_cols] = num_imputer.transform(X_test[numeric_cols])

# Use most frequent value for categorical features
if len(categorical_cols) > 0:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    X_train[categorical_cols] = cat_imputer.fit_transform(X_train[categorical_cols])
    X_val[categorical_cols] = cat_imputer.transform(X_val[categorical_cols])
    X_test[categorical_cols] = cat_imputer.transform(X_test[categorical_cols])

# 3. Apply undersampling strategy - set non-default:default = 3:1
print("\nApplying undersampling strategy...")
print(f"Training set distribution before undersampling: {np.bincount(y_train)}")

# Calculate target sampling ratio - majority:minority = 3:1
sampling_strategy = {0: int(np.sum(y_train == 1) * 3), 1: np.sum(y_train == 1)}
print(f"Undersampling target ratio: Negative:{sampling_strategy[0]}, Positive:{sampling_strategy[1]}")

# Apply random undersampling
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

print(f"Training set shape after undersampling: {X_train_resampled.shape}")
print(f"Training set distribution after undersampling: {np.bincount(y_train_resampled)}")
print(f"New class ratio: 1:{np.sum(y_train_resampled == 0)/np.sum(y_train_resampled == 1):.2f}")

# 4. Model Training - Using undersampled data
print("\nStarting model training...")

# 4.1 Base XGBoost model initialization
base_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    # Use smaller scale_pos_weight since we've already balanced data via undersampling
    scale_pos_weight=1.0,  # Can try values between 1.0-1.5
    random_state=42,
    # The following parameters help with imbalanced datasets
    max_delta_step=1,  # Helps with imbalanced data
    min_child_weight=1  # Lower values allow algorithm to learn more specific patterns
)

# 4.2 Hyperparameter search space - Focus on improving prediction for default class
param_distributions = {
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 400, 500]
}

# 4.3 Set cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4.4 Custom scorer - Emphasize recall
# beta=2 makes recall weight twice as much as precision
def f_beta_scorer(y_true, y_pred_proba, beta=2.0):
    """Custom F-beta score, emphasizing recall more"""
    y_pred = (y_pred_proba >= 0.3).astype(int)  # Lower threshold favors finding default customers
    
    # Calculate recall and precision
    true_pos = np.sum((y_true == 1) & (y_pred == 1))
    pred_pos = np.sum(y_pred == 1)
    actual_pos = np.sum(y_true == 1)
    
    if pred_pos == 0 or actual_pos == 0:
        return 0.0
    
    precision = true_pos / pred_pos
    recall = true_pos / actual_pos
    
    # Calculate F-beta score, beta>1 emphasizes recall more
    f_beta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + 1e-10)
    return f_beta

# Make scorer compatible with sklearn
from sklearn.metrics import make_scorer
f_beta_scorer_func = make_scorer(f_beta_scorer, greater_is_better=True, needs_proba=True)

# 4.5 Hyperparameter search
print("Starting hyperparameter optimization...")
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=30,  # Increase number of iterations for better results
    scoring=f_beta_scorer_func,  # Use custom F-beta score
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Train using undersampled data
random_search.fit(X_train_resampled, y_train_resampled)

# 4.6 Output best parameters and score
print("\nHyperparameter optimization results:")
print(f"Best F-beta score: {random_search.best_score_:.4f}")
print(f"Best parameters: {random_search.best_params_}")

# 4.7 Train final model with best parameters
print("\nTraining final model with best parameters...")
best_model = XGBClassifier(**random_search.best_params_, random_state=42)
# Simplify training call to avoid version compatibility issues
best_model.fit(X_train_resampled, y_train_resampled)

# 5. Find optimal classification threshold - Focus on improving recall
print("\nFinding optimal classification threshold...")
# Generate prediction probabilities on validation set
y_val_pred_proba = best_model.predict_proba(X_val)[:, 1]

# Calculate precision and recall for different thresholds
precision, recall, thresholds = precision_recall_curve(y_val, y_val_pred_proba)

# Calculate F-beta scores for different thresholds (beta=2, emphasize recall)
beta = 2
f_beta_scores = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + 1e-10)

# Find threshold with best F-beta score
best_idx = np.argmax(f_beta_scores)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
best_f_beta = f_beta_scores[best_idx]

print(f"Best threshold: {best_threshold:.4f}, F{beta}-score: {best_f_beta:.4f}")
print(f"At this threshold - Precision: {precision[best_idx]:.4f}, Recall: {recall[best_idx]:.4f}")

# Visualize precision-recall curve
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, marker='.', label='PR curve')
plt.scatter(recall[best_idx], precision[best_idx], c='red', label=f'Best threshold: {best_threshold:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve with Best Threshold (F2-score optimized)')
plt.legend()
plt.grid(True)
plt.savefig('precision_recall_curve_undersampling.png')  # Save the chart
plt.close()

# 6. Model Evaluation
print("\nModel Evaluation:")
# Function to predict using custom threshold
def predict_with_threshold(model, X, threshold=0.5):
    """Make predictions using custom threshold"""
    y_pred_proba = model.predict_proba(X)[:, 1]
    return (y_pred_proba >= threshold).astype(int)

# 6.1 Training set evaluation
y_train_pred = predict_with_threshold(best_model, X_train_resampled, best_threshold)
y_train_pred_proba = best_model.predict_proba(X_train_resampled)[:, 1]

print("\nTraining set evaluation (undersampled dataset):")
print(classification_report(y_train_resampled, y_train_pred))
print(f"Training set ROC-AUC: {roc_auc_score(y_train_resampled, y_train_pred_proba):.4f}")
print(f"Training set F1 score: {f1_score(y_train_resampled, y_train_pred):.4f}")
print(f"Training set F{beta} score: {f_beta_scores[best_idx]:.4f}")

# 6.2 Evaluate on original training set
y_orig_train_pred = predict_with_threshold(best_model, X_train, best_threshold)
y_orig_train_pred_proba = best_model.predict_proba(X_train)[:, 1]

print("\nOriginal training set evaluation (using best threshold):")
print(classification_report(y_train, y_orig_train_pred))
print(f"Original training set ROC-AUC: {roc_auc_score(y_train, y_orig_train_pred_proba):.4f}")
print(f"Original training set F1 score: {f1_score(y_train, y_orig_train_pred):.4f}")

# 6.3 Validation set evaluation
y_val_pred = predict_with_threshold(best_model, X_val, best_threshold)
print("\nValidation set evaluation (using best threshold):")
print(classification_report(y_val, y_val_pred))
print(f"Validation set ROC-AUC: {roc_auc_score(y_val, y_val_pred_proba):.4f}")
print(f"Validation set F1 score: {f1_score(y_val, y_val_pred):.4f}")

# 6.4 Test set evaluation (if labels are available)
if y_test is not None:
    y_test_pred = predict_with_threshold(best_model, X_test, best_threshold)
    y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]
    
    print("\nTest set evaluation (using best threshold):")
    print(classification_report(y_test, y_test_pred))
    print(f"Test set ROC-AUC: {roc_auc_score(y_test, y_test_pred_proba):.4f}")
    print(f"Test set F1 score: {f1_score(y_test, y_test_pred):.4f}")

# 6.5 Confusion matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_val, y_val_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Validation Set Confusion Matrix (using best threshold)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('confusion_matrix_undersampling.png')  # Save the chart
plt.close()

# 6.6 Feature importance
plt.figure(figsize=(12, 10))
feature_importance = best_model.feature_importances_
sorted_idx = np.argsort(feature_importance)
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx])
plt.yticks(range(len(sorted_idx)), np.array(X_train.columns)[sorted_idx])
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.savefig('feature_importance_undersampling.png')  # Save the chart
plt.close()

# 7. Export prediction results
if y_test is None and test_loan_ids is not None:
    # If test set has no labels, create predictions
    test_pred = predict_with_threshold(best_model, X_test, best_threshold)
    test_pred_proba = best_model.predict_proba(X_test)[:, 1]
    
    # Create submission file
    submission = pd.DataFrame({
        'LoanID': test_loan_ids,
        'Default': test_pred,
        'DefaultProbability': test_pred_proba
    })
    
    submission.to_csv('loan_default_predictions_undersampling.csv', index=False)
    print("\nPrediction results exported to 'loan_default_predictions_undersampling.csv'")

# 8. SHAP Value Analysis
print("\nPerforming SHAP value analysis...")

# Create explainer object - TreeExplainer is optimized for tree-based models like XGBoost
explainer = shap.TreeExplainer(best_model)

# Calculate SHAP values for validation set
# Limit to a subset if the dataset is very large
sample_size = min(500, X_val.shape[0])  # Use at most 500 samples to avoid memory issues
X_val_sample = X_val.sample(sample_size, random_state=42) if X_val.shape[0] > sample_size else X_val
shap_values = explainer.shap_values(X_val_sample)

# 8.1 Summary plot - This creates the visualization like in your first image
plt.figure(figsize=(12, 16))
shap.summary_plot(
    shap_values, 
    X_val_sample,
    plot_type="dot", 
    show=False,  # Don't display immediately 
    max_display=20  # Show top 20 features
)
plt.tight_layout()
plt.savefig('shap_summary_plot.png', bbox_inches='tight', dpi=300)
plt.close()

# 8.2 Create bar plot of mean absolute SHAP values for top features
plt.figure(figsize=(12, 8))
shap.summary_plot(
    shap_values, 
    X_val_sample,
    plot_type="bar", 
    show=False,
    max_display=10  # Top 10 features
)
plt.tight_layout()
plt.savefig('shap_importance_bar_plot.png', bbox_inches='tight', dpi=300)
plt.close()

# 8.3 Create detailed dependency plots for top 5 features
# Get mean absolute SHAP values for each feature
mean_abs_shap = np.abs(shap_values).mean(0)
feature_importance_order = np.argsort(mean_abs_shap)[::-1]  # Sort in descending order
top_features = feature_importance_order[:5]  # Get indices of top 5 features

# Create separate dependency plots for each top feature
for i, feature_idx in enumerate(top_features):
    feature_name = X_val.columns[feature_idx]
    plt.figure(figsize=(10, 6))
    shap.dependence_plot(
        feature_idx, 
        shap_values, 
        X_val_sample,
        show=False,
        interaction_index=None  # Set to None to turn off interaction coloring
    )
    plt.title(f"SHAP Dependence Plot for {feature_name}")
    plt.tight_layout()
    plt.savefig(f'shap_dependence_{i+1}_{feature_name}.png', bbox_inches='tight', dpi=300)
    plt.close()

# 8.4 Create a partial dependence plot for the most important feature (similar to your second image)
top_feature_idx = feature_importance_order[0]
top_feature_name = X_val.columns[top_feature_idx]

plt.figure(figsize=(10, 6))
shap_values_for_top_feature = shap_values[:, top_feature_idx]
feature_values = X_val_sample.iloc[:, top_feature_idx]

# Sort points by feature value
sorted_indices = np.argsort(feature_values)
sorted_values = feature_values.iloc[sorted_indices]
sorted_shap = shap_values_for_top_feature[sorted_indices]

plt.scatter(sorted_values, sorted_shap, alpha=0.6)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.7)
plt.xlabel(top_feature_name)
plt.ylabel(f'SHAP value (impact on model output)')
plt.title(f'Impact of {top_feature_name} on Prediction')
plt.tight_layout()
plt.savefig('top_feature_dependence_plot.png', bbox_inches='tight', dpi=300)
plt.close()

# 8.5 Print top 10 most important features based on SHAP
print("\nTop 10 most important features based on SHAP values:")
for i in range(10):
    if i < len(feature_importance_order):
        feature_idx = feature_importance_order[i]
        feature_name = X_val.columns[feature_idx]
        feature_importance = mean_abs_shap[feature_idx]
        print(f"{i+1}. {feature_name}: {feature_importance:.4f}")

print("\nSHAP analysis completed and visualizations saved!")
print("\nModel training and evaluation complete!")

Loading data...
Training set shape: (255347, 18)
Test set shape: (109435, 17)

Data preprocessing...
Checking missing values...
Training set missing values:
LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64

Test set missing values:
LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
dtype: int64

Target variable distribution:
Non-default (0): 88.39%
Default (1): 11.61%
Positive samples